# Module 2: Drawing & Color Spaces

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/02_Drawing_and_Color.ipynb)

**Week 3-4: Drawing Primitives & Color Transformations**

## Learning Objectives
- Draw shapes, lines, and text on images
- Understand different color spaces (BGR, RGB, HSV, LAB)
- Convert between color spaces
- Apply thresholding techniques

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

---
## Mathematical Foundations: Geometry and Color Theory

Before working with drawing and colors, let's establish the **theoretical mathematical foundations**.

### Topics Covered
1. **Coordinate Systems and Geometry**: Points, lines, curves in discrete space
2. **Parametric Representations**: Mathematical description of shapes
3. **Color Spaces**: Mathematical models of color
4. **Color Transformations**: Linear algebra of color conversion
5. **Thresholding Theory**: Piecewise functions and segmentation

### 1. Coordinate Systems and Discrete Geometry

#### Image Coordinate System

Images use a **discrete coordinate system** where:
- Origin $(0, 0)$ is at the **top-left corner**
- $x$-axis increases **rightward** (columns)
- $y$-axis increases **downward** (rows)
- Coordinates are **integers**: $(x, y) \in \mathbb{Z}^2$

**Coordinate transformation** from Cartesian to Image coordinates:
$$
\begin{bmatrix} x_{img} \\ y_{img} \end{bmatrix} = 
\begin{bmatrix} x_{cart} \\ h - y_{cart} \end{bmatrix}
$$

Where $h$ is the image height.

#### Point Representation

A **point** in 2D space is represented as:
$$
\mathbf{p} = \begin{bmatrix} x \\ y \end{bmatrix} \in \mathbb{Z}^2
$$

In **homogeneous coordinates** (for transformations):
$$
\mathbf{p}_h = \begin{bmatrix} x \\ y \\ 1 \end{bmatrix} \in \mathbb{R}^3
$$

#### Line Representation

A **line** from point $\mathbf{p}_1 = (x_1, y_1)$ to $\mathbf{p}_2 = (x_2, y_2)$ can be represented:

1. **Parametric form**:
   $$
   \mathbf{p}(t) = (1-t)\mathbf{p}_1 + t\mathbf{p}_2, \quad t \in [0, 1]
   $$
   $$
   \begin{cases}
   x(t) = (1-t)x_1 + tx_2 = x_1 + t(x_2 - x_1) \\
   y(t) = (1-t)y_1 + ty_2 = y_1 + t(y_2 - y_1)
   \end{cases}
   $$

2. **Implicit form** (line equation):
   $$
   ax + by + c = 0
   $$
   Where $a = y_2 - y_1$, $b = x_1 - x_2$, $c = x_2y_1 - x_1y_2$

3. **Slope-intercept form**:
   $$
   y = mx + b
   $$
   Where $m = \frac{y_2 - y_1}{x_2 - x_1}$ (slope), $b$ is $y$-intercept

In [ ]:
# Demonstrate parametric line representation
print("=" * 70)
print("PARAMETRIC LINE REPRESENTATION")
print("=" * 70)

# Define two points
p1 = np.array([50, 100])
p2 = np.array([200, 250])

print(f"\nPoint 1: p1 = {p1}")
print(f"Point 2: p2 = {p2}")

# Generate points along the line using parametric form
print("\nParametric form: p(t) = (1-t)·p1 + t·p2, t ∈ [0,1]")
print("\nSample points:")
t_values = [0, 0.25, 0.5, 0.75, 1.0]
points = []
for t in t_values:
    pt = (1-t) * p1 + t * p2
    points.append(pt)
    print(f"  t={t:.2f}: p({t:.2f}) = ({pt[0]:.1f}, {pt[1]:.1f})")

# Calculate line equation (implicit form)
a = p2[1] - p1[1]
b = p1[0] - p2[0]
c = p2[0]*p1[1] - p1[0]*p2[1]

print(f"\nImplicit form: ax + by + c = 0")
print(f"  a = {a:.1f}, b = {b:.1f}, c = {c:.1f}")
print(f"  Line equation: {a:.1f}x + {b:.1f}y + {c:.1f} = 0")

# Slope-intercept form
if p2[0] != p1[0]:
    slope = (p2[1] - p1[1]) / (p2[0] - p1[0])
    intercept = p1[1] - slope * p1[0]
    print(f"\nSlope-intercept form: y = mx + b")
    print(f"  m (slope) = {slope:.4f}")
    print(f"  b (intercept) = {intercept:.4f}")
    print(f"  Equation: y = {slope:.4f}x + {intercept:.4f}")

# Visualize
canvas = np.zeros((300, 300, 3), dtype=np.uint8)

# Draw the line
cv2.line(canvas, tuple(p1), tuple(p2), (0, 255, 0), 2)

# Mark the sample points
for i, (t, pt) in enumerate(zip(t_values, points)):
    cv2.circle(canvas, (int(pt[0]), int(pt[1])), 5, (0, 0, 255), -1)
    cv2.putText(canvas, f't={t:.2f}', (int(pt[0])+10, int(pt[1])-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

# Mark endpoints
cv2.circle(canvas, tuple(p1), 8, (255, 0, 0), -1)
cv2.circle(canvas, tuple(p2), 8, (255, 0, 0), -1)
cv2.putText(canvas, 'p1', (p1[0]-20, p1[1]-10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
cv2.putText(canvas, 'p2', (p2[0]+10, p2[1]+10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
plt.title('Parametric Line: p(t) = (1-t)·p1 + t·p2', fontsize=14)
plt.axis('off')
plt.show()

print("\nKey Insight: Lines are linear interpolations between two points!")

### 2. Parametric Shapes and Curves

#### Circle

A **circle** with center $(c_x, c_y)$ and radius $r$ can be defined:

1. **Implicit equation**:
   $$
   (x - c_x)^2 + (y - c_y)^2 = r^2
   $$

2. **Parametric form** (using angle $\theta \in [0, 2\pi]$):
   $$
   \begin{cases}
   x(\theta) = c_x + r\cos(\theta) \\
   y(\theta) = c_y + r\sin(\theta)
   \end{cases}
   $$

**Distance formula**: Point $(x, y)$ is on the circle if:
$$
d = \sqrt{(x - c_x)^2 + (y - c_y)^2} = r
$$

#### Ellipse

An **ellipse** with center $(c_x, c_y)$, semi-major axis $a$, semi-minor axis $b$, and rotation angle $\alpha$:

**Parametric form**:
$$
\begin{bmatrix} x(\theta) \\ y(\theta) \end{bmatrix} = 
\begin{bmatrix} c_x \\ c_y \end{bmatrix} +
\begin{bmatrix}
\cos\alpha & -\sin\alpha \\
\sin\alpha & \cos\alpha
\end{bmatrix}
\begin{bmatrix} a\cos\theta \\ b\sin\theta \end{bmatrix}
$$

Expanded:
$$
\begin{cases}
x(\theta) = c_x + a\cos\theta\cos\alpha - b\sin\theta\sin\alpha \\
y(\theta) = c_y + a\cos\theta\sin\alpha + b\sin\theta\cos\alpha
\end{cases}
$$

#### Rectangle

A **rectangle** with top-left corner $(x_1, y_1)$ and bottom-right corner $(x_2, y_2)$:

**Vertices** (in order):
$$
\mathbf{v}_1 = \begin{bmatrix} x_1 \\ y_1 \end{bmatrix}, \quad
\mathbf{v}_2 = \begin{bmatrix} x_2 \\ y_1 \end{bmatrix}, \quad
\mathbf{v}_3 = \begin{bmatrix} x_2 \\ y_2 \end{bmatrix}, \quad
\mathbf{v}_4 = \begin{bmatrix} x_1 \\ y_2 \end{bmatrix}
$$

**Geometric properties**:
- Width: $w = |x_2 - x_1|$
- Height: $h = |y_2 - y_1|$
- Area: $A = w \times h$
- Perimeter: $P = 2(w + h)$
- Center: $\mathbf{c} = \begin{bmatrix} \frac{x_1+x_2}{2} \\ \frac{y_1+y_2}{2} \end{bmatrix}$

In [ ]:
# Demonstrate parametric shapes
print("=" * 70)
print("PARAMETRIC SHAPE REPRESENTATIONS")
print("=" * 70)

# Circle parameters
center = (150, 150)
radius = 80

print(f"\nCIRCLE:")
print(f"Center: c = {center}")
print(f"Radius: r = {radius}")
print(f"Implicit equation: (x-{center[0]})² + (y-{center[1]})² = {radius}²")
print(f"Parametric: x(θ) = {center[0]} + {radius}cos(θ)")
print(f"            y(θ) = {center[1]} + {radius}sin(θ)")

# Generate circle points using parametric form
theta = np.linspace(0, 2*np.pi, 100)
x_circle = center[0] + radius * np.cos(theta)
y_circle = center[1] + radius * np.sin(theta)

# Verify points are on circle
distances = np.sqrt((x_circle - center[0])**2 + (y_circle - center[1])**2)
print(f"\nVerification: All points at distance r from center?")
print(f"  Mean distance: {np.mean(distances):.2f}")
print(f"  Expected (r): {radius}")
print(f"  Error: {np.abs(np.mean(distances) - radius):.6f}")

# Ellipse parameters
a = 100  # semi-major
b = 60   # semi-minor
alpha = np.radians(45)  # rotation angle

print(f"\nELLIPSE:")
print(f"Center: c = {center}")
print(f"Semi-major axis: a = {a}")
print(f"Semi-minor axis: b = {b}")
print(f"Rotation angle: α = {np.degrees(alpha):.1f}°")

# Generate ellipse points
theta_ellipse = np.linspace(0, 2*np.pi, 100)
# Rotation matrix
R = np.array([[np.cos(alpha), -np.sin(alpha)],
              [np.sin(alpha), np.cos(alpha)]])

ellipse_points = []
for t in theta_ellipse:
    # Point on canonical ellipse
    p_canonical = np.array([a * np.cos(t), b * np.sin(t)])
    # Rotate and translate
    p_rotated = R @ p_canonical
    p_final = p_rotated + np.array(center)
    ellipse_points.append(p_final)

ellipse_points = np.array(ellipse_points)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Circle
canvas1 = np.zeros((300, 300, 3), dtype=np.uint8)
cv2.circle(canvas1, center, radius, (0, 255, 0), 2)
# Mark some parametric points
for angle in [0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi]:
    x = int(center[0] + radius * np.cos(angle))
    y = int(center[1] + radius * np.sin(angle))
    cv2.circle(canvas1, (x, y), 5, (0, 0, 255), -1)
    cv2.putText(canvas1, f'{np.degrees(angle):.0f}deg', (x+10, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

cv2.circle(canvas1, center, 5, (255, 0, 0), -1)
axes[0].imshow(cv2.cvtColor(canvas1, cv2.COLOR_BGR2RGB))
axes[0].set_title(f'Circle: (x-{center[0]})² + (y-{center[1]})² = {radius}²', fontsize=12)
axes[0].axis('off')

# Ellipse
canvas2 = np.zeros((300, 300, 3), dtype=np.uint8)
cv2.ellipse(canvas2, center, (a, b), np.degrees(alpha), 0, 360, (0, 255, 0), 2)
# Mark center and axes
cv2.circle(canvas2, center, 5, (255, 0, 0), -1)
# Draw axes
axis1_end = (int(center[0] + a*np.cos(alpha)), int(center[1] + a*np.sin(alpha)))
axis2_end = (int(center[0] - b*np.sin(alpha)), int(center[1] + b*np.cos(alpha)))
cv2.line(canvas2, center, axis1_end, (255, 255, 0), 1)
cv2.line(canvas2, center, axis2_end, (0, 255, 255), 1)

axes[1].imshow(cv2.cvtColor(canvas2, cv2.COLOR_BGR2RGB))
axes[1].set_title(f'Ellipse: a={a}, b={b}, α={np.degrees(alpha):.1f}°', fontsize=12)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Shapes are defined by parametric equations!")

### 3. Color Space Theory

#### Color as a Vector

A **color** is represented as a vector in a 3D color space:
$$
\mathbf{c} = \begin{bmatrix} c_1 \\ c_2 \\ c_3 \end{bmatrix} \in \mathbb{R}^3
$$

Different color spaces use different basis vectors and coordinate systems.

#### RGB Color Space

**RGB** (Red, Green, Blue) is an **additive** color model:
$$
\mathbf{c}_{RGB} = \begin{bmatrix} R \\ G \\ B \end{bmatrix}, \quad R, G, B \in [0, 255] \text{ (8-bit)}
$$

Or normalized: $R, G, B \in [0, 1]$

**Color mixing** (linear combination):
$$
\mathbf{c} = \alpha \mathbf{c}_1 + (1-\alpha) \mathbf{c}_2, \quad \alpha \in [0, 1]
$$

**Grayscale conversion** (weighted sum):
$$
L = 0.299R + 0.587G + 0.114B
$$

These weights come from **human luminance perception**.

#### HSV Color Space

**HSV** (Hue, Saturation, Value) uses a **cylindrical** coordinate system:
$$
\mathbf{c}_{HSV} = \begin{bmatrix} H \\ S \\ V \end{bmatrix}
$$

Where:
- **H (Hue)**: Angle around the color wheel, $H \in [0°, 360°]$ (in OpenCV: $[0, 179]$)
- **S (Saturation)**: Distance from center, $S \in [0, 1]$ or $[0, 255]$
- **V (Value)**: Brightness, $V \in [0, 1]$ or $[0, 255]$

**RGB to HSV Conversion**:

Let $R, G, B \in [0, 1]$ (normalized):

$$
V = \max(R, G, B)
$$

$$
S = \begin{cases}
0 & \text{if } V = 0 \\
\frac{V - \min(R,G,B)}{V} & \text{otherwise}
\end{cases}
$$

$$
H = \begin{cases}
0° & \text{if } S = 0 \\
60° \times \frac{G - B}{V - \min(R,G,B)} & \text{if } V = R \\
60° \times \left(2 + \frac{B - R}{V - \min(R,G,B)}\right) & \text{if } V = G \\
60° \times \left(4 + \frac{R - G}{V - \min(R,G,B)}\right) & \text{if } V = B
\end{cases}
$$

If $H < 0$, add $360°$.

In [ ]:
# Demonstrate color space mathematics
print("=" * 70)
print("COLOR SPACE THEORY AND TRANSFORMATIONS")
print("=" * 70)

# Define a color in RGB
rgb_color = np.array([180, 100, 220], dtype=np.uint8)  # Purple
rgb_normalized = rgb_color / 255.0

print("\nRGB COLOR REPRESENTATION:")
print(f"RGB (8-bit): {rgb_color}")
print(f"RGB (normalized): {rgb_normalized}")
print(f"\nColor vector in R³:")
print(f"c_RGB = [{rgb_normalized[0]:.3f}, {rgb_normalized[1]:.3f}, {rgb_normalized[2]:.3f}]ᵀ")

# Grayscale conversion
print("\n" + "="*70)
print("GRAYSCALE CONVERSION (Luminance)")
print("="*70)
print("\nFormula: L = 0.299R + 0.587G + 0.114B")
L = 0.299 * rgb_normalized[2] + 0.587 * rgb_normalized[1] + 0.114 * rgb_normalized[0]
print(f"L = 0.299×{rgb_normalized[2]:.3f} + 0.587×{rgb_normalized[1]:.3f} + 0.114×{rgb_normalized[0]:.3f}")
print(f"L = {L:.3f}")
print(f"L (8-bit) = {int(L * 255)}")

# Verify with OpenCV
test_img = np.full((1, 1, 3), rgb_color, dtype=np.uint8)
gray_cv = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)[0, 0]
print(f"OpenCV grayscale: {gray_cv}")
print(f"Match: {np.isclose(int(L * 255), gray_cv, atol=1)}")

# RGB to HSV conversion (manual calculation)
print("\n" + "="*70)
print("RGB TO HSV CONVERSION")
print("="*70)

R, G, B = rgb_normalized[2], rgb_normalized[1], rgb_normalized[0]  # Note BGR->RGB
print(f"\nInput RGB (normalized): R={R:.3f}, G={G:.3f}, B={B:.3f}")

# Calculate V
V = max(R, G, B)
print(f"\nV = max(R, G, B) = max({R:.3f}, {G:.3f}, {B:.3f}) = {V:.3f}")

# Calculate S
min_rgb = min(R, G, B)
if V == 0:
    S = 0
else:
    S = (V - min_rgb) / V
print(f"\nS = (V - min(R,G,B)) / V")
print(f"  = ({V:.3f} - {min_rgb:.3f}) / {V:.3f}")
print(f"  = {S:.3f}")

# Calculate H
if S == 0:
    H = 0
else:
    if V == R:
        H = 60 * (G - B) / (V - min_rgb)
        branch = "V = R"
    elif V == G:
        H = 60 * (2 + (B - R) / (V - min_rgb))
        branch = "V = G"
    else:  # V == B
        H = 60 * (4 + (R - G) / (V - min_rgb))
        branch = "V = B"
    
    if H < 0:
        H += 360

print(f"\nH calculation (branch: {branch}):")
print(f"  H = {H:.1f}°")

print(f"\nFinal HSV: H={H:.1f}°, S={S:.3f}, V={V:.3f}")

# Verify with OpenCV
hsv_cv = cv2.cvtColor(test_img, cv2.COLOR_BGR2HSV)[0, 0]
print(f"\nOpenCV HSV (H in [0,179], S,V in [0,255]):")
print(f"  H={hsv_cv[0]}, S={hsv_cv[1]}, V={hsv_cv[2]}")
print(f"  H (converted to degrees): {hsv_cv[0] * 2}°")
print(f"  S (normalized): {hsv_cv[1]/255:.3f}")
print(f"  V (normalized): {hsv_cv[2]/255:.3f}")

# Visualize color in different spaces
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# RGB cube visualization (simplified)
rgb_display = np.full((100, 100, 3), rgb_color, dtype=np.uint8)
axes[0].imshow(cv2.cvtColor(rgb_display, cv2.COLOR_BGR2RGB))
axes[0].set_title(f'RGB Color\nR={rgb_color[2]}, G={rgb_color[1]}, B={rgb_color[0]}', fontsize=12)
axes[0].axis('off')

# Grayscale
gray_display = np.full((100, 100), gray_cv, dtype=np.uint8)
axes[1].imshow(gray_display, cmap='gray')
axes[1].set_title(f'Grayscale\nL={gray_cv}', fontsize=12)
axes[1].axis('off')

# HSV representation
axes[2].text(0.5, 0.7, f'HSV Representation', ha='center', va='center',
             fontsize=14, fontweight='bold', transform=axes[2].transAxes)
axes[2].text(0.5, 0.5, f'H = {H:.1f}° (Hue)', ha='center', va='center',
             fontsize=12, transform=axes[2].transAxes)
axes[2].text(0.5, 0.35, f'S = {S:.3f} (Saturation)', ha='center', va='center',
             fontsize=12, transform=axes[2].transAxes)
axes[2].text(0.5, 0.2, f'V = {V:.3f} (Value)', ha='center', va='center',
             fontsize=12, transform=axes[2].transAxes)
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Color spaces are different coordinate systems in R³!")

### 4. Thresholding as Piecewise Functions

#### Binary Thresholding

**Thresholding** is a **piecewise function** that maps pixel intensities to binary values:

$$
g(x, y) = \begin{cases}
\text{maxVal} & \text{if } f(x, y) > \text{thresh} \\
0 & \text{otherwise}
\end{cases}
$$

Where:
- $f(x, y)$: Input pixel intensity at $(x, y)$
- $g(x, y)$: Output pixel value
- $\text{thresh}$: Threshold value
- $\text{maxVal}$: Maximum value (usually 255)

#### Threshold Types

1. **Binary**:
   $$
   g = \begin{cases} \text{maxVal} & \text{if } f > T \\ 0 & \text{otherwise} \end{cases}
   $$

2. **Binary Inverted**:
   $$
   g = \begin{cases} 0 & \text{if } f > T \\ \text{maxVal} & \text{otherwise} \end{cases}
   $$

3. **Truncate**:
   $$
   g = \begin{cases} T & \text{if } f > T \\ f & \text{otherwise} \end{cases}
   $$

4. **To Zero**:
   $$
   g = \begin{cases} f & \text{if } f > T \\ 0 & \text{otherwise} \end{cases}
   $$

5. **To Zero Inverted**:
   $$
   g = \begin{cases} 0 & \text{if } f > T \\ f & \text{otherwise} \end{cases}
   $$

#### Otsu's Method

**Otsu's method** finds the optimal threshold $T^*$ by maximizing **between-class variance**:

$$
T^* = \arg\max_{T} \sigma_B^2(T)
$$

Where the between-class variance is:
$$
\sigma_B^2(T) = \omega_0(T) \omega_1(T) [\mu_0(T) - \mu_1(T)]^2
$$

With:
- $\omega_0(T) = \sum_{i=0}^{T} p_i$: Probability of class 0 (background)
- $\omega_1(T) = \sum_{i=T+1}^{255} p_i$: Probability of class 1 (foreground)
- $\mu_0(T)$: Mean intensity of class 0
- $\mu_1(T)$: Mean intensity of class 1
- $p_i$: Probability of intensity level $i$ (from histogram)

This is equivalent to minimizing **within-class variance**:
$$
\sigma_W^2(T) = \omega_0(T)\sigma_0^2(T) + \omega_1(T)\sigma_1^2(T)
$$

In [ ]:
# Demonstrate thresholding mathematics
print("=" * 70)
print("THRESHOLDING THEORY: PIECEWISE FUNCTIONS")
print("=" * 70)

# Create a simple gradient for demonstration
x = np.linspace(0, 255, 256)
threshold = 127

# Define threshold functions
def binary_thresh(x, T, maxVal=255):
    return np.where(x > T, maxVal, 0)

def binary_inv_thresh(x, T, maxVal=255):
    return np.where(x > T, 0, maxVal)

def truncate_thresh(x, T):
    return np.where(x > T, T, x)

def tozero_thresh(x, T):
    return np.where(x > T, x, 0)

def tozero_inv_thresh(x, T):
    return np.where(x > T, 0, x)

# Apply all threshold types
y_binary = binary_thresh(x, threshold)
y_binary_inv = binary_inv_thresh(x, threshold)
y_truncate = truncate_thresh(x, threshold)
y_tozero = tozero_thresh(x, threshold)
y_tozero_inv = tozero_inv_thresh(x, threshold)

# Plot the piecewise functions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original
axes[0, 0].plot(x, x, 'b-', linewidth=2)
axes[0, 0].axvline(threshold, color='r', linestyle='--', label=f'T={threshold}')
axes[0, 0].set_title('Original: g(x) = x', fontsize=12)
axes[0, 0].set_xlabel('Input intensity')
axes[0, 0].set_ylabel('Output intensity')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Binary
axes[0, 1].plot(x, y_binary, 'b-', linewidth=2)
axes[0, 1].axvline(threshold, color='r', linestyle='--', label=f'T={threshold}')
axes[0, 1].set_title('Binary:\ng = maxVal if x>T else 0', fontsize=12)
axes[0, 1].set_xlabel('Input intensity')
axes[0, 1].set_ylabel('Output intensity')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# Binary Inverted
axes[0, 2].plot(x, y_binary_inv, 'b-', linewidth=2)
axes[0, 2].axvline(threshold, color='r', linestyle='--', label=f'T={threshold}')
axes[0, 2].set_title('Binary Inv:\ng = 0 if x>T else maxVal', fontsize=12)
axes[0, 2].set_xlabel('Input intensity')
axes[0, 2].set_ylabel('Output intensity')
axes[0, 2].grid(True, alpha=0.3)
axes[0, 2].legend()

# Truncate
axes[1, 0].plot(x, y_truncate, 'b-', linewidth=2)
axes[1, 0].axvline(threshold, color='r', linestyle='--', label=f'T={threshold}')
axes[1, 0].set_title('Truncate:\ng = T if x>T else x', fontsize=12)
axes[1, 0].set_xlabel('Input intensity')
axes[1, 0].set_ylabel('Output intensity')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# To Zero
axes[1, 1].plot(x, y_tozero, 'b-', linewidth=2)
axes[1, 1].axvline(threshold, color='r', linestyle='--', label=f'T={threshold}')
axes[1, 1].set_title('To Zero:\ng = x if x>T else 0', fontsize=12)
axes[1, 1].set_xlabel('Input intensity')
axes[1, 1].set_ylabel('Output intensity')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

# To Zero Inverted
axes[1, 2].plot(x, y_tozero_inv, 'b-', linewidth=2)
axes[1, 2].axvline(threshold, color='r', linestyle='--', label=f'T={threshold}')
axes[1, 2].set_title('To Zero Inv:\ng = 0 if x>T else x', fontsize=12)
axes[1, 2].set_xlabel('Input intensity')
axes[1, 2].set_ylabel('Output intensity')
axes[1, 2].grid(True, alpha=0.3)
axes[1, 2].legend()

plt.tight_layout()
plt.show()

print("\nMathematical formulas for each threshold type:")
print("\nBinary:         g = maxVal if f > T else 0")
print("Binary Inv:     g = 0 if f > T else maxVal")
print("Truncate:       g = T if f > T else f")
print("To Zero:        g = f if f > T else 0")
print("To Zero Inv:    g = 0 if f > T else f")

print("\nKey Insight: Thresholding creates piecewise constant or linear functions!")

### Summary: Mathematical Foundations for Drawing & Color

#### 1. Geometric Primitives
- **Points**: Vectors in $\mathbb{Z}^2$ or $\mathbb{R}^3$ (homogeneous)
- **Lines**: Parametric form $\mathbf{p}(t) = (1-t)\mathbf{p}_1 + t\mathbf{p}_2$
- **Circles**: $(x-c_x)^2 + (y-c_y)^2 = r^2$ or $x = c_x + r\cos\theta$, $y = c_y + r\sin\theta$
- **Ellipses**: Rotated parametric form with rotation matrix

#### 2. Color Spaces
- **RGB**: Additive model, $\mathbf{c} = [R, G, B]^T \in \mathbb{R}^3$
- **Grayscale**: $L = 0.299R + 0.587G + 0.114B$ (luminance)
- **HSV**: Cylindrical coordinates $(H, S, V)$
- **Color mixing**: Linear combinations $\alpha \mathbf{c}_1 + (1-\alpha)\mathbf{c}_2$

#### 3. Thresholding
- **Piecewise functions**: Map intensities to discrete classes
- **Otsu's method**: Optimal threshold via variance maximization
- **Five types**: Binary, Binary-Inv, Truncate, ToZero, ToZero-Inv

#### Key Formulas Table

| Concept | Formula | OpenCV Function |
|---------|---------|----------------|
| Line (parametric) | $\mathbf{p}(t) = (1-t)\mathbf{p}_1 + t\mathbf{p}_2$ | `cv2.line()` |
| Circle (implicit) | $(x-c_x)^2 + (y-c_y)^2 = r^2$ | `cv2.circle()` |
| Ellipse (parametric) | $x = c_x + a\cos\theta\cos\alpha - b\sin\theta\sin\alpha$ | `cv2.ellipse()` |
| RGB to Gray | $L = 0.299R + 0.587G + 0.114B$ | `cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)` |
| HSV Value | $V = \max(R, G, B)$ | `cv2.cvtColor(img, cv2.COLOR_BGR2HSV)` |
| Binary Threshold | $g = \begin{cases} 255 & f > T \\ 0 & \text{else} \end{cases}$ | `cv2.threshold(img, T, 255, cv2.THRESH_BINARY)` |
| Otsu's Threshold | $T^* = \arg\max_T \sigma_B^2(T)$ | `cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)` |

---

**Next**: Apply these mathematical foundations to practical drawing and color operations!

## 2.1 Drawing Lines

In [ ]:
# Create blank canvas
canvas = np.zeros((400, 600, 3), dtype=np.uint8)

# Draw line: cv2.line(img, pt1, pt2, color, thickness)
cv2.line(canvas, (50, 50), (550, 50), (0, 255, 0), 2)  # Green horizontal
cv2.line(canvas, (50, 100), (550, 300), (255, 0, 0), 3)  # Blue diagonal
cv2.line(canvas, (300, 50), (300, 350), (0, 0, 255), 4)  # Red vertical

# Anti-aliased line
cv2.line(canvas, (50, 200), (550, 200), (255, 255, 0), 2, cv2.LINE_AA)

plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
plt.title('Drawing Lines')
plt.axis('off')
plt.show()

## 2.2 Drawing Shapes

In [ ]:
canvas = np.zeros((500, 700, 3), dtype=np.uint8)

# Rectangle: cv2.rectangle(img, pt1, pt2, color, thickness)
cv2.rectangle(canvas, (50, 50), (200, 150), (0, 255, 0), 2)  # Outline
cv2.rectangle(canvas, (250, 50), (400, 150), (0, 255, 0), -1)  # Filled

# Circle: cv2.circle(img, center, radius, color, thickness)
cv2.circle(canvas, (100, 300), 50, (255, 0, 0), 2)  # Outline
cv2.circle(canvas, (300, 300), 50, (255, 0, 0), -1)  # Filled

# Ellipse: cv2.ellipse(img, center, axes, angle, startAngle, endAngle, color, thickness)
cv2.ellipse(canvas, (550, 100), (80, 40), 0, 0, 360, (0, 0, 255), 2)
cv2.ellipse(canvas, (550, 100), (80, 40), 45, 0, 180, (0, 255, 255), -1)

# Polygon
pts = np.array([[500, 250], [600, 250], [650, 350], [550, 400], [450, 350]], np.int32)
pts = pts.reshape((-1, 1, 2))
cv2.polylines(canvas, [pts], True, (255, 255, 0), 3)

plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
plt.title('Drawing Shapes')
plt.axis('off')
plt.show()

## 2.3 Adding Text

In [ ]:
canvas = np.zeros((400, 800, 3), dtype=np.uint8)

# Different fonts
fonts = [
    (cv2.FONT_HERSHEY_SIMPLEX, 'SIMPLEX'),
    (cv2.FONT_HERSHEY_PLAIN, 'PLAIN'),
    (cv2.FONT_HERSHEY_DUPLEX, 'DUPLEX'),
    (cv2.FONT_HERSHEY_COMPLEX, 'COMPLEX'),
    (cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 'SCRIPT')
]

y_pos = 50
for font, name in fonts:
    cv2.putText(canvas, name, (50, y_pos), font, 1, (0, 255, 255), 2, cv2.LINE_AA)
    y_pos += 60

# Different sizes
cv2.putText(canvas, 'Small', (400, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
cv2.putText(canvas, 'Medium', (400, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
cv2.putText(canvas, 'Large', (400, 300), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)

plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
plt.title('Text Rendering')
plt.axis('off')
plt.show()

## 2.4 Color Spaces - BGR vs RGB vs HSV

In [ ]:
# Load sample image
!wget -q https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/481px-Cat03.jpg -O sample.jpg
img = cv2.imread('sample.jpg')

# Convert to different color spaces
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Display
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original (BGR→RGB)')

axes[0, 1].imshow(hsv)
axes[0, 1].set_title('HSV')

axes[0, 2].imshow(lab)
axes[0, 2].set_title('LAB')

axes[1, 0].imshow(gray, cmap='gray')
axes[1, 0].set_title('Grayscale')

# Show individual HSV channels
h, s, v = cv2.split(hsv)
axes[1, 1].imshow(s, cmap='gray')
axes[1, 1].set_title('Saturation Channel')

axes[1, 2].imshow(v, cmap='gray')
axes[1, 2].set_title('Value Channel')

for ax in axes.flat:
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2.5 Color Detection using HSV

In [ ]:
# Convert to HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define color range (red in this example)
# HSV ranges: H: 0-179, S: 0-255, V: 0-255
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

# Create masks
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
mask = mask1 + mask2

# Apply mask
result = cv2.bitwise_and(img, img, mask=mask)

# Display
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original')
axes[1].imshow(mask, cmap='gray')
axes[1].set_title('Red Color Mask')
axes[2].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
axes[2].set_title('Red Objects Only')
for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2.6 Thresholding

In [ ]:
# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Simple threshold
_, thresh1 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# Otsu's thresholding
_, thresh2 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Adaptive thresholding
thresh3 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                 cv2.THRESH_BINARY, 11, 2)
thresh4 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                 cv2.THRESH_BINARY, 11, 2)

# Display
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(gray, cmap='gray')
axes[0, 0].set_title('Original Grayscale')

axes[0, 1].imshow(thresh1, cmap='gray')
axes[0, 1].set_title('Simple Threshold (127)')

axes[0, 2].imshow(thresh2, cmap='gray')
axes[0, 2].set_title("Otsu's Threshold")

axes[1, 0].imshow(thresh3, cmap='gray')
axes[1, 0].set_title('Adaptive Mean')

axes[1, 1].imshow(thresh4, cmap='gray')
axes[1, 1].set_title('Adaptive Gaussian')

axes[1, 2].axis('off')

for ax in axes.flat[:-1]:
    ax.axis('off')
plt.tight_layout()
plt.show()

## Project: Simple Drawing App

In [ ]:
# Create a simple drawing canvas
canvas = np.ones((600, 800, 3), dtype=np.uint8) * 255

# Add title
cv2.putText(canvas, 'My Digital Canvas', (250, 50), 
            cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 0), 3)

# Draw a sun
cv2.circle(canvas, (650, 100), 40, (0, 255, 255), -1)  # Yellow sun

# Draw a house
cv2.rectangle(canvas, (100, 300), (350, 500), (0, 100, 200), -1)  # House body
pts = np.array([[100, 300], [225, 200], [350, 300]], np.int32)
cv2.fillPoly(canvas, [pts], (0, 0, 200))  # Roof
cv2.rectangle(canvas, (180, 380), (270, 500), (100, 50, 0), -1)  # Door
cv2.circle(canvas, (250, 440), 5, (255, 255, 0), -1)  # Doorknob

# Draw a tree
cv2.rectangle(canvas, (480, 400), (520, 500), (0, 100, 50), -1)  # Trunk
cv2.circle(canvas, (500, 350), 70, (0, 200, 0), -1)  # Leaves

# Draw ground
cv2.rectangle(canvas, (0, 500), (800, 600), (0, 150, 0), -1)

plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
plt.title('My Drawing')
plt.axis('off')
plt.show()

# Save the drawing
cv2.imwrite('my_drawing.png', canvas)
print("Drawing saved!")

## Summary

You learned:
- Drawing primitives (lines, circles, rectangles, polygons)
- Text rendering with different fonts
- Color space conversions (BGR, RGB, HSV, LAB)
- Color-based object detection using HSV
- Various thresholding techniques

**Next**: Module 3 - Filtering & Enhancement